In [1]:
from data_generator import vis_train_features
from IPython.display import Markdown, display
from data_generator import vis_train_features, plot_raw_audio
from IPython.display import Audio
from sample_models import *
from train_utils import train_model
%matplotlib inline

# extract label and audio features for a single training example
vis_text, vis_raw_audio, vis_mfcc_feature, vis_spectrogram_feature, vis_audio_path = vis_train_features()

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
There are 11126 total training examples.


In [2]:
import json
import os
import numpy as np
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dropout, TimeDistributed, Dense, Activation, Embedding
from char_model.model import build_model, load_weights
from char_model.sample import build_sample_model, sample,sample2


In [3]:
model_final = final_model(input_dim=161,
                          filters=200,
                          kernel_size=11, 
                          conv_stride=2,
                          conv_border_mode='valid',
                          output_dim=83,
                          number_of_layers=2,
                          units=200)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
the_input (InputLayer)       (None, None, 161)         0         
_________________________________________________________________
layer_1_conv (Conv1D)        (None, None, 200)         354400    
_________________________________________________________________
conv_batch_norm (BatchNormal (None, None, 200)         800       
_________________________________________________________________
rnn_1 (GRU)                  (None, None, 200)         240600    
_________________________________________________________________
bt_rnn_1 (BatchNormalization (None, None, 200)         800       
_________________________________________________________________
final_layer_of_rnn (GRU)     (None, None, 200)         240600    
___________________________

In [ ]:
# train_model(input_to_softmax=model_final, 
#             pickle_path='model_final.pickle', 
#             save_model_path='model_final.h5', 
#             spectrogram=True)# change to False if you would like to use MFCC features

In [ ]:
# import record_audio as rd
# rec = rd.Recorder(channels=2)
# with rec.open('114.wav', 'wb') as recfile:
#     recfile.record(duration=5.0)

In [4]:
Audio('001.wav')

In [12]:
import numpy as np
from data_generator import AudioGenerator
from keras import backend as K
from utils import int_sequence_to_text
from IPython.display import Audio

def get_predictions(input_to_softmax, model_path):
    """ Print a model's decoded predictions
    Params:
        index (int): The example you would like to visualize
        partition (str): One of 'train' or 'validation'
        input_to_softmax (Model): The acoustic model
        model_path (str): Path to saved acoustic model's weights
    """
    # load the train and test data
    data_gen = AudioGenerator()
    data_gen.load_train_data()
    #data_gen.load_validation_data()
    
    # obtain the true transcription and the audio features 
    audio_path = '001.wav'
    data_point = data_gen.normalize(data_gen.featurize(audio_path))
        
    # obtain and decode the acoustic model's predictions
    input_to_softmax.load_weights(model_path)
    prediction = input_to_softmax.predict(np.expand_dims(data_point, axis=0))
    output_length = [input_to_softmax.output_length(data_point.shape[0])] 
    pred_ints = (K.eval(K.ctc_decode(
                prediction, output_length)[0][0])+1).flatten().tolist()
    
    # play the audio file, and display the true and predicted transcriptions
    print('-'*80)
    Audio(audio_path)
    #print('True transcription:\n' + '\n' + transcr)
    print('-'*80)
    print('Predicted transcription:\n' + '\n' + ''.join(int_sequence_to_text(pred_ints)))
    print('-'*80)

In [5]:
import numpy as np
from data_generator import AudioGenerator
from keras import backend as K
from utils import int_sequence_to_text
from IPython.display import Audio

def get_predictions_one(input_to_softmax, model_path):
    """ Print a model's decoded predictions
    Params:
        index (int): The example you would like to visualize
        partition (str): One of 'train' or 'validation'
        input_to_softmax (Model): The acoustic model
        model_path (str): Path to saved acoustic model's weights
    """
    # load the train and test data
    data_gen = AudioGenerator(spectrogram=True)
    data_gen.load_train_data()
    #data_gen.load_validation_data()
   
    audio_path = '001.wav'
    data_point = data_gen.normalize(data_gen.featurize(audio_path))

        
    # obtain and decode the acoustic model's predictions
    input_to_softmax.load_weights(model_path)
    
    prediction = input_to_softmax.predict(np.expand_dims(data_point, axis=0))
    output_length = [input_to_softmax.output_length(data_point.shape[0])] 
    
    pred_ints = (K.eval(K.ctc_decode(
                prediction, output_length)[0][0])+1).flatten().tolist()
    print(pred_ints)
    print(output_length)
    # play the audio file, and display the true and predicted transcriptions
    print('-'*80)
    print('Predicted transcription:\n' + '\n' + ''.join(int_sequence_to_text(pred_ints)))
    print('-'*80)
    pre = int_sequence_to_text(pred_ints)
    lan_pre=sample(100, pre, len(pre)+1)
    print(lan_pre)

In [13]:
get_predictions(input_to_softmax=model_final, model_path='results/test_final.h5')

--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Predicted transcription:

টুলি আমাল প্রথম ঘলেমাসা
--------------------------------------------------------------------------------
